Curve fitting
=============

The general case
----------------
In many cases of data analysis we want to estimate some parameters.
In other words, we have a *model* of the data which depends on
a set of parameters, say $a_1,\dots,a_M$ which we want to fit
to our data points $\{x_i,y_i,u_i\}, i=1\dots N$.
Here, $u_i$ is the (measured or estimated) uncertainty in $y_i$;
we assume that the uncertainties in $x_i$ are small. This is often a good
assumption; if the uncertainties in $x_i$ are *not* small, then the fitting
procedure is a *lot* messier. 

We can express our model as
$$
y(x)=f(x\,|\,a_1,\dots,a_M)
$$ 
How do we proceed?

Let us rephrase things a little. Suppose we are given a set of
parameters $\{a\}$. What is the probability that we obtain the 
data $y(x_i)=f(x_i\,|\,a_1,\dots,a_M)$? *Given* a probabilty density function (pdf) $p_y(y)$ which 
describes this probability, and *assuming* that the measurements
are independent, the probability of finding each $y(x_i)$ in $(y,y+\delta y)$
is
$$
P(\text{data}\,|\,\text{model})=\prod_{i=1}^N\,p_y(y(x_i))\,\delta y
$$
(We read this as "the probability of the data given the model").
In the absence of further information we *assume* that $p_y$
is given by a normal distribution,
$$
p_y(y(x_i))=\frac{1}{\sqrt{2\pi}\sigma_i}e^{-(f(x_i\,|\,\{a\})-y_i)^2/2\sigma_i^2}
$$
We *assume* that the standard deviations of the pdfs are given
by the experimental uncertainties, $\sigma_i=u_i$.

Now, what we actually want is not $P(\text{data}\,|\,\text{model})$
but $P(\text{model}\,|\,\text{data})$. We would like to maximise this
latter quantity by adjusting the model parameters.
Fortunately, we know that Bayes' theorem tells us that
$P(\text{model}\,|\,\text{data})\propto P(\text{data}\,|\,\text{model})P(\text{model})$.
If we know *nothing* about $P(\text{model})$, we can *assume*
that is a flat distribution --- all parameters are likely, a priori.
(This is known as an uninformative prior in the game).
The result is that maximising $P(\text{model}\,|\,\text{data})$ is equivalent
to maximising $P(\text{data}\,|\,\text{model})$, i.e. maximising
$$
P(\text{data}\,|\,\text{model})\propto\prod_{i=1}^N\,e^{-(f(x_i|\{a\})-y_i)^2/2\sigma_i^2}\,\delta y
$$
or, taking the negative of the log of this, minimising
$$
-N\log\delta y+\left[\sum_{i=1}^N\,\frac{[y_i-f(x_i)]^2}{2u_i^2}\right]
$$
Thus we can find the ``best-fit'' parameters by minimising
$$
\chi^2=\sum_{i=1}^N\,\frac{[y_i-f(x_i\,|\,\{a\})]^2}{u_i^2}
$$
i.e. we minimise the sum of the squares of the weighted difference between
model and data. This is known colloquially as least-squares or
chi-squared fitting.

Statistics tells us that under all those assumptions above, 
$\chi^2$ is distributed according to the chi-squared distribution
of $N-M$ degrees of freedom, $\chi^2_{N-M}(x)$, with expectation value $N-M$.
Thus we expect that $\chi^2/(N-M)$, the reduced chi-squared, has a value
close to one. We will take this as an indication of a "good fit".

We still have to *minimise* $\chi^2$. If our model is simple,
this can be done analytically, e.g for the straight line
$y=ax+b$ (see, for example, the lab manual). In general, however,
we will be forced to use a non-linear optimisation routine on the
computer. In these notes we will give an example of the use
of the Levenberg-Marquardt method available in `scipy`.


Fitting a straight line
-----------------------

We have seen that finding the best-fit parameters is equivalent to
minimising the statistic
$$
\chi^2=\sum_{i=1}^N\,\frac{[y_i-f(x_i\,|\,\{a\})]^2}{u_i^2}
%%\chi^2=\sum_{i=1}^N\frac{(y_i-y_\text{th}(i,a_1,a_2,\dots))^2}{e_i^2}.
$$


For a small class of problems, in particular where $f(x_i\,|\,\{a\})$
is
a polynomial with unknown coefficients and the uncertainties are only in $y$,
these equations can be solved. We consider the simplest non-trivial
case, the straight line, $y=f(x_i\,|\,\{a\})=a_1x+a_2\equiv ax+b$. 

If the uncertainty in each point is the same, this is
known as linear regression.
You might find that your pocket calculator can do this.
In most uses in physics, however, the uncertainty differs from point to point -
the data are said to be *heteroskedastic*. We will use 
*weighted least squares* minimisation to handle this.


The minimisation is done with respect to the parameters.
At the minimum in chisquare, i.e. for the best-fit parameters,
the partial derivative of the chisquared with respect to the parameters 
vanishes.
$$
\begin{aligned}
\frac{\partial \chi^2}{\partial a} &= 0 \\
&=-2\sum\frac{x_i(y_i-ax_i-b)}{u_i^2} \\
&=-2\sum\frac{x_iy_i}{u_i^2}
+2a\sum\frac{x_i^2}{u_i^2}
+2b\sum\frac{x_i}{u_i^2}
\end{aligned}
$$
and
$$
\begin{aligned}
\frac{\partial \chi^2}{\partial b} &= 0 \\
&= -2\sum\frac{y_i-ax_i-b}{u_i^2} \\
&=-2\sum\frac{y_i}{u_i^2}
+2a\sum\frac{x_i}{u_i^2}
+2b\sum\frac{1}{u_i^2}.
\end{aligned}
$$
These equations can be simplified by introducing
$$
\begin{aligned}
S=\sum\frac{1}{u_i^2},
\qquad
S_x&=\sum\frac{x_i}{u_i^2},
\qquad
S_y=\sum\frac{y_i}{u_i^2},\\
S_{xx}=\sum\frac{x_i^2}{u_i^2},
\quad&\quad
S_{xy}=\sum\frac{x_iy_i}{u_i^2}.
\end{aligned}
$$
Thus, for the best-fit parameters we have:
$$
%\begin{aligned}
-2S_{xy}+2aS_{xx}+2bS_x=0\quad\quad
-2S_{y}+2aS_{x}+2bS=0
%\end{aligned}
$$
and hence
$$
%\begin{aligned}
aS_{xx}+bS_{yy}=S_{xy}\qquad\qquad
aS_{x}+bS=S_{y}.
%\end{aligned}
$$
Then the parameters are given by  the solution of these equations:
$$
\begin{aligned}
a&=\frac{SS_{xy}-S_xS_y}{\Delta}\\
b&=\frac{S_yS_{xx}-S_xS_{xy}}{\Delta}
\end{aligned}
$$
where $\Delta=SS_{xx}-S_x^2$.

Using the usual formula for propagation of uncertainties, the variances of
$a$ and $b$ can be found:
$$
\begin{aligned}
s_a^2&=\frac{S_{xx}}{\Delta}\\
s_b^2&=\frac{S}{\Delta}\\
\textrm{Cov}(a,b)&=\frac{-S_x}{\Delta}
\end{aligned}
$$

How do we know that the pair of parameters $a,b$ represent a
good fit to the data? This requires a further statistical test of the
significance of the chisquare found at the position of least squares.
However a simple rule of thumb is that $\chi/\sqrt{N-2}$ should be of
the order of 1. (This assumes that the uncertainties are normally distributed
and properly evaluated).

